In [1]:
import pandas as  pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.svm import SVC,SVR
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier,RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,RandomForestRegressor
from xgboost import XGBRegressor
import time

In [6]:
def just_give_df(v):     # v is my data frame
    col=v.columns        # storing all column in my a variable
    
    
#***************************** checking for nan in each and every column **********************************


    print(v.isna().sum())     # checking weather any nan are there are not
    for i in col:
        if v[i].isna().sum()==0:
            print("no nan")
        elif v[i].dtypes=="object":
            v[i].fillna(method=mode()[0],inplace=True)
            print("nan filled with backword fill")
        elif v[i].dtypes!="object":
            v[i].fillna(v[i].mean(),inplace=True)
    print("__________________________________________")
    from scipy.stats import norm
    sns.boxplot(v)
    plt.show()
    print("______________________________________________")
#********************************** checking outliers by using boxplot *******************************************
    for i in col:
        if v[i].dtypes!= "object":
            plt.subplot(1,2,1)
            plt.hist(v[i], bins=20)
            plt.show()
            plt.subplot(1,2,2)
            sns.distplot(v[i],fit=norm)
            plt.show()
            print(i,"  : numarical column ")
            if np.round(v[i].skew())==0:
                print("mean standard deviation method for  : ",i)
                m =v[i].mean()
                s =v[i].std()
                low = m-(3*s)
                up = m+(3*s)
                print()
                print("OUTLIERS for column is  : ",i)
                print("____________________________________________")
                print()
                z=v[(v[i]<low)|(v[i]>up)]
                print(z)
                print("__________________________________________________")
                    
#**************************** check for trimming or capping****************************************************

                if z[i].size>=v[i].sample(frac=0.50).size:
                    print("*************************************************")
                    print('-------  CAPING -------')
                    v[i]=v[i].map(lambda x: low if x<=low else up if x>=up else x)
                    print("Outliers after capping  : ". v[(v[i]<low)|(v[i]>up)])
                else :
                    print("-------Trimming--------")
                    v = v[(v[i]>=low)&(v[i]<=up)]
                    print("outliers after trimming  : ", v[(v[i]<low)|(v[i]>up)])
                    print()
                    print("_______________________________________________________")
                    print()
            elif np.round(v[i].skew())!=1:
                print("using IQR method for  : ")
                q1 = v[i].quantile(0.25)
                q3 = v[i].quantile(0.75)
                iqr = q3 - q1
                lower = q1 - (1.5 * iqr)
                upper = q3 + (1.5 * iqr)
                print()
                print("OUTLIERS for column is :  ",i)
                print("____________________________________________")
                print()
                z=v[  (v[i] < lower) | (v[i] > upper)  ]
                print(z)
                    
#**************************** check for trimming or capping****************************************************

                if z[i].size>=v[i].sample(frac=0.50).size:
                    print("*************************************************")
                    print('-------  CAPING -------')
                    v[i]=v[i].map(lambda x: low if x<=low else up if x>=up else x)
                    print("Outliers after capping  : ", v[(v[i]<low)|(v[i]>up)])
                else :
                    print("-------Trimming--------")
                    v = v[(v[i]>=low)&(v[i]<=up)]
                    print("outliers after trimming  : ", v[(v[i]<low)|(v[i]>up)])
                        
        elif v[i].dtypes=="object":
            sns.countplot(x=i,data=v)
            plt.show()
            print(i,"  : catagorical column")
# **************************** suppuration of dependent and independent columns ******************************** 
    print()
    X=v.drop(v.columns[-1],axis=1)  
    y=v[v.columns[-1]]
    from sklearn.preprocessing import LabelEncoder
    k=X.columns
#************************************* encoding ********************************************
    print(k)
    for i in k:
        if X[i].dtypes=="object":
            X=pd.get_dummies(X)
            print("printing indipendent X after encoding")
            print("________________________")
            print(X)
            break
            print()
        else  :
            print("all indipendent columns are numarical ")
    if y.dtypes=="object":
        le=LabelEncoder()
        le.fit(y)
        le.transform(y)
        y=le.transform(y)
        print("printing dependent y after encoding")
        print("______________________")
        print(y)
        print()
    else :
        print("dependent column in numarical data")
    print("___________________________________________________")
    print()
#***************************** imbalancing ******************************************
    print("check for imbalancing")
    print(v.value_counts(v[v.columns[-1]]))
    if v.dtypes[-1]=="object":
        from imblearn.over_sampling import RandomOverSampler
        ros=RandomOverSampler()
        X_ros,y_ros=ros.fit_resample(X,y)           # balanced data stored is X_ros,y_ros
        print()
        print(y.size,X.size,X_ros,y_ros)        
        print("________________________________________________")
        print()
        print("--------------performing classification-------------")
        print()
# ******************************** performing classifiers **********************************
        reslist=[]
        for r in range(0,10):
            Xtrain, Xtest, ytrain, ytest = train_test_split(X_ros, y_ros, test_size=0.20, random_state= r)
            dtree=DecisionTreeClassifier()
            svm=SVC()
            log=LogisticRegression()
            kn=KNeighborsClassifier()
            gra=GradientBoostingClassifier()
            ada=AdaBoostClassifier()
            ran=RandomForestClassifier()
            reslist=[]
            engg=[dtree,svm,log,kn,gra,ada,ran]
            for m in engg:
                m.fit(Xtrain,ytrain)
                a=m.score(Xtest,ytest)
                reslist.append(a)
            print("Random State Number : ",r)
            print()
            print("Accuracy List : ",reslist)
            print()
            print(  engg[ reslist.index(max(reslist)) ], end = " ==> ")
            print(max(reslist))
            print("***********************************************")
            print()
        
        
    elif v.dtypes[-1]!="object":
        print("in dependenrt column is in numarical no need to do imbalancing")
    
# ********************************** importing regression **********************************

        
        print("----------------performing regression method--------------------")
        for r in range(0,10):
            Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.20, random_state= r)
            dtree=DecisionTreeRegressor()
            svm=SVR()
            kn=KNeighborsRegressor()
            gra=GradientBoostingRegressor()
            ada=AdaBoostRegressor()
            ran=RandomForestRegressor()
            xg=XGBRegressor()
            lreg = LinearRegression()
            reslist = []
            engg=[dtree,svm,kn,gra,ada,ran,xg, lreg]
            for m in engg:
                m.fit(Xtrain,ytrain)
                a=m.score(Xtest,ytest)
                reslist.append(a)
            print("Random State Number : ",r)
            print()
            print("Accuracy List : ",reslist)
            print()
            print(  engg[ reslist.index(max(reslist)) ], end = " ==> ")
            print(max(reslist))
            print("***********************************************")
            print()